In [2]:
pip install Flask tensorflow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import io

app = Flask(__name__)
model = load_model('landmark_recognition_model.h5')

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['file']
    img = Image.open(file.stream).resize((224, 224))  # Resize if needed
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])

    return jsonify({'predicted_class': int(predicted_class)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.7:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:192.168.1.7 - - [03/May/2025 13:55:29] "GET /predict HTTP/1.1" 405 -
INFO:werkzeug:192.168.1.7 - - [03/May/2025 13:55:29] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:192.168.1.7 - - [03/May/2025 13:55:33] "GET / HTTP/1.1" 404 -
INFO:werkzeug:192.168.1.7 - - [03/May/2025 13:55:38] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/May/2025 13:58:45] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/May/2025 13:58:45] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:192.168.1.7 - - [03/May/2025 14:09:53] "GET / HTTP/1.1" 404 -
INFO:werkzeug:192.168.1.7 - - [03/May/2025 14:09:53] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:192.168.1.7 - - [03/May/2025 14:10:01] "GET /predict HTTP/1.1"

In [8]:
from flask import Flask, request, jsonify
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import io

# Initialize the Flask application
app = Flask(__name__)

# Load the trained model
model = load_model('landmark_recognition_model.h5')  # Adjust the path if needed

# Department label map
label_map = {
    0: "BusinessDepartment",
    1: "CivilDepartment",
    2: "EEDepartment",
    3: "Library",
    4: "NewCSDepartment",
    5: "OldCSDepartment"
}

def load_and_preprocess(img_path):
    """
    Function to load and preprocess image for MobileNetV2 model.
    """
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocess image for MobileNetV2
    return img_array

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Retrieve the image file from the request
        file = request.files['file']

        # Save the image temporarily for processing
        img = Image.open(file.stream)
        img_path = 'temp_image.jpg'
        img.save(img_path)

        # Preprocess the image
        img_array = load_and_preprocess(img_path)

        # Predict using the loaded model
        predictions = model.predict(img_array)

        # Get the class with the highest probability
        predicted_class = np.argmax(predictions[0])

        # Map the predicted class index to the corresponding label
        predicted_label = label_map.get(predicted_class, "Unknown")

        # Return the prediction in JSON format
        return jsonify({'predicted_class': predicted_class, 'predicted_label': predicted_label})

    except Exception as e:
        return jsonify({'error': str(e)}), 500  # Return error if something goes wrong

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.7:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:192.168.1.11 - - [03/May/2025 14:29:30] "POST /predict HTTP/1.1" 500 -


In [2]:
from flask import Flask, request, jsonify
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
import math
import tempfile
import base64
from PIL import Image
import io

app = Flask(__name__)

# Global reference building sizes data
REFERENCE_SIZES = {
    'BusinessDepartment': {'height': 15.0, 'width': 40.0, 'reference_pixel_height': 450},
    'Library': {'height': 11.0, 'width': 45.0, 'reference_pixel_height': 450},
    'EEDepartment': {'height': 12.0, 'width': 50.0, 'reference_pixel_height': 450},
    'CivilDepartment': {'height': 14.0, 'width': 50.0, 'reference_pixel_height': 500},
    'OldCSDepartment': {'height': 15.0, 'width': 48.0, 'reference_pixel_height': 450},
    'NewCSDepartment': {'height': 16.0, 'width': 50.0, 'reference_pixel_height': 500}
}

# Building coordinates for location estimation
BUILDING_COORDINATES = {
    'BusinessDepartment': (100, 200),
    'EEDepartment': (280, 400),
    'Library': (300, 150),
    'OldCSDepartment': (250, 350),
    'CivilDepartment': (150, 300),
    'NewCSDepartment': (400, 250)
}

# Global model variables (will load on first request)
recognition_model = None
label_encoder = None
base_model = None


def load_recognition_model():
    """Loads the trained landmark recognition model"""
    global recognition_model, label_encoder, base_model
    
    # Check if already loaded
    if recognition_model is not None:
        return recognition_model, label_encoder, base_model
    
    # Paths - adjust as needed for your deployment
    model_path = 'landmark_recognition_model.h5'
    annotations_path = "annotations.csv"
    
    # Load model
    recognition_model = load_model(model_path)
    
    # Load annotations CSV
    df = pd.read_csv(annotations_path)
    
    # Encode labels
    label_encoder = LabelEncoder()
    label_encoder.fit(df['building_name'])
    
    # Get base model for feature extraction
    base_model = tf.keras.applications.MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3),
        pooling='avg'
    )
    base_model.trainable = False
    
    return recognition_model, label_encoder, base_model


def load_and_preprocess_image(img_path_or_data):
    """Load and preprocess image for the model from file path or image data"""
    if isinstance(img_path_or_data, str):  # If path
        img = image.load_img(img_path_or_data, target_size=(224, 224))
    else:  # If image data
        img = Image.open(io.BytesIO(img_path_or_data)).resize((224, 224))
        
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array, img


def recognize_landmark(img_data, model, label_encoder, base_model):
    """Recognize landmark in the given image"""
    # Preprocess image
    preprocessed_img, original_img = load_and_preprocess_image(img_data)
    
    # Extract features
    features = base_model.predict(preprocessed_img)
    
    # Predict building
    prediction = model.predict(features)
    predicted_class = np.argmax(prediction[0])
    building_name = label_encoder.classes_[predicted_class]
    confidence = float(prediction[0][predicted_class])
    
    return building_name, confidence, original_img


def detect_building_boundaries(img_data):
    """
    Detect the boundaries of a building in the image
    Returns the bounding box coordinates and the building's height in pixels
    """
    # Save image data to a temporary file if it's not already a file path
    if not isinstance(img_data, str):
        temp = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
        temp.write(img_data)
        temp.close()
        img_path = temp.name
    else:
        img_path = img_data
    
    # Read image
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Could not read image")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Edge detection using Canny
    edges = cv2.Canny(blurred, 50, 150)
    
    # Dilate edges to connect broken lines
    dilated = cv2.dilate(edges, np.ones((3, 3), np.uint8), iterations=2)
    
    # Find contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the largest contour (assuming it's the building)
    if not contours:
        return None, 0, img_rgb
    
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Get bounding box
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Clean up temporary file if created
    if not isinstance(img_data, str):
        os.unlink(img_path)
    
    return (x, y, w, h), h, img_rgb


def estimate_distance_by_size(building_name, pixel_height):
    """
    Estimate the distance to a building based on its apparent size in pixels
    compared to a reference size.
    """
    if building_name not in REFERENCE_SIZES:
        return None
    
    # Get reference measurements for this building
    reference_data = REFERENCE_SIZES[building_name]
    reference_height_m = reference_data['height']
    reference_pixel_height = reference_data['reference_pixel_height']
    
    # Calculate focal length from reference data
    # Assumes reference image was taken at a known distance (e.g., 10 meters)
    reference_distance = 10.0  # meters
    focal_length = (reference_pixel_height * reference_distance) / reference_height_m
    
    # Calculate distance using the focal length and current pixel height
    estimated_distance = (reference_height_m * focal_length) / pixel_height
    
    return estimated_distance


def estimate_triangulation_distance(img_data1, img_data2, baseline_distance):
    """
    Estimate distance using triangulation from two images taken at known distance apart
    """
    # Save image data to temporary files
    temp1 = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
    temp1.write(img_data1)
    temp1.close()
    
    temp2 = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
    temp2.write(img_data2)
    temp2.close()
    
    # Detect building in both images
    bbox1, height1, _ = detect_building_boundaries(temp1.name)
    bbox2, height2, _ = detect_building_boundaries(temp2.name)
    
    # Clean up temporary files
    os.unlink(temp1.name)
    os.unlink(temp2.name)
    
    if bbox1 is None or bbox2 is None:
        return None
    
    # Calculate image centers (width)
    img1 = cv2.imread(temp1.name)
    img2 = cv2.imread(temp2.name)
    img_center1 = img1.shape[1] / 2
    img_center2 = img2.shape[1] / 2
    
    # Calculate building centers in both images
    building_center1 = bbox1[0] + bbox1[2]/2
    building_center2 = bbox2[0] + bbox2[2]/2
    
    # Calculate parallax (difference in position)
    parallax1 = building_center1 - img_center1
    parallax2 = building_center2 - img_center2
    
    # Calculate change in parallax
    parallax_change = abs(parallax1 - parallax2)
    
    if parallax_change == 0:
        # Avoid division by zero
        return None
    
    # Calculate distance using triangulation formula
    # Distance = (baseline * focal_length) / parallax_change
    # Assuming focal_length is approximately image_width for simplicity
    focal_length = img1.shape[1]
    distance = (baseline_distance * focal_length) / parallax_change
    
    return distance


def estimate_location(building_name, distance, bearing=None):
    """
    Estimate user's location based on distance to a landmark and optional bearing
    """
    if building_name not in BUILDING_COORDINATES:
        return None
    
    building_x, building_y = BUILDING_COORDINATES[building_name]
    
    if bearing is None:
        # Without bearings, default to south of the building
        angle_rad = math.radians(180)
    else:
        # Convert bearings to radians for calculation
        angle_rad = math.radians(bearing)
    
    # Calculate position
    x = building_x - distance * math.sin(angle_rad)
    y = building_y + distance * math.cos(angle_rad)
    
    return x, y


def get_image_from_request():
    """Extract image data from request (base64 or file upload)"""
    if 'file' in request.files:
        # File upload
        file = request.files['file']
        return file.read()
    elif 'image' in request.json:
        # Base64 encoded image
        base64_data = request.json['image']
        # Remove the data URI prefix if present (e.g., "data:image/jpeg;base64,")
        if ',' in base64_data:
            base64_data = base64_data.split(',', 1)[1]
        return base64.b64decode(base64_data)
    else:
        raise ValueError("No image data found in request")


@app.route('/api/recognize', methods=['POST'])
def api_recognize():
    """API endpoint for landmark recognition"""
    try:
        # Load models if not already loaded
        global recognition_model, label_encoder, base_model
        if recognition_model is None:
            recognition_model, label_encoder, base_model = load_recognition_model()
        
        # Get image data from request
        img_data = get_image_from_request()
        
        # Recognize landmark
        building_name, confidence, _ = recognize_landmark(img_data, recognition_model, label_encoder, base_model)
        
        return jsonify({
            'success': True,
            'building_name': building_name,
            'confidence': confidence
        })
    
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 400


@app.route('/api/estimate-distance', methods=['POST'])
def api_estimate_distance():
    """API endpoint for distance estimation"""
    try:
        # Load models if not already loaded
        global recognition_model, label_encoder, base_model
        if recognition_model is None:
            recognition_model, label_encoder, base_model = load_recognition_model()
        
        # Get image data from request
        img_data = get_image_from_request()
        
        # Recognize landmark
        building_name, confidence, _ = recognize_landmark(img_data, recognition_model, label_encoder, base_model)
        
        # Detect building boundaries
        bbox, pixel_height, _ = detect_building_boundaries(img_data)
        
        if bbox is None:
            return jsonify({
                'success': False,
                'error': 'Could not detect building boundaries'
            }), 400
        
        # Estimate distance
        distance = estimate_distance_by_size(building_name, pixel_height)
        
        if distance is None:
            return jsonify({
                'success': False,
                'error': f'No reference data available for {building_name}'
            }), 400
        
        # Estimate location if bearing provided
        location = None
        if 'bearing' in request.json:
            bearing = float(request.json['bearing'])
            user_x, user_y = estimate_location(building_name, distance, bearing)
            location = {'x': user_x, 'y': user_y}
        
        return jsonify({
            'success': True,
            'building_name': building_name,
            'confidence': confidence,
            'distance': distance,
            'location': location
        })
    
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 400


@app.route('/api/triangulate', methods=['POST'])
def api_triangulate():
    """API endpoint for triangulation-based distance estimation"""
    try:
        # Validate request
        if 'image1' not in request.json or 'image2' not in request.json or 'baseline' not in request.json:
            return jsonify({
                'success': False,
                'error': 'Missing required parameters: image1, image2, and baseline'
            }), 400
        
        # Get image data from request
        img_data1 = base64.b64decode(request.json['image1'])
        img_data2 = base64.b64decode(request.json['image2'])
        baseline = float(request.json['baseline'])
        
        # Estimate distance using triangulation
        distance = estimate_triangulation_distance(img_data1, img_data2, baseline)
        
        if distance is None:
            return jsonify({
                'success': False,
                'error': 'Triangulation failed'
            }), 400
        
        return jsonify({
            'success': True,
            'distance': distance
        })
    
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 400


@app.route('/api/location', methods=['POST'])
def api_location():
    """API endpoint for user location estimation"""
    try:
        # Validate request
        if 'building_name' not in request.json or 'distance' not in request.json:
            return jsonify({
                'success': False,
                'error': 'Missing required parameters: building_name and distance'
            }), 400
        
        building_name = request.json['building_name']
        distance = float(request.json['distance'])
        bearing = None
        if 'bearing' in request.json:
            bearing = float(request.json['bearing'])
        
        # Estimate location
        location = estimate_location(building_name, distance, bearing)
        
        if location is None:
            return jsonify({
                'success': False,
                'error': f'Could not estimate location for {building_name}'
            }), 400
        
        return jsonify({
            'success': True,
            'x': location[0],
            'y': location[1],
            'building_coordinates': BUILDING_COORDINATES[building_name]
        })
    
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 400


@app.route('/api/all-in-one', methods=['POST'])
def api_all_in_one():
    """Comprehensive API endpoint that performs recognition, distance estimation, and location in one call"""
    try:
        # Load models if not already loaded
        global recognition_model, label_encoder, base_model
        if recognition_model is None:
            recognition_model, label_encoder, base_model = load_recognition_model()
        
        # Get image data from request
        img_data = get_image_from_request()
        
        # Recognize landmark
        building_name, confidence, _ = recognize_landmark(img_data, recognition_model, label_encoder, base_model)
        
        # Detect building boundaries
        bbox, pixel_height, _ = detect_building_boundaries(img_data)
        
        if bbox is None:
            return jsonify({
                'success': False,
                'error': 'Could not detect building boundaries'
            }), 400
        
        # Estimate distance
        distance = estimate_distance_by_size(building_name, pixel_height)
        
        if distance is None:
            return jsonify({
                'success': False,
                'error': f'No reference data available for {building_name}'
            }), 400
        
        # Estimate location if bearing provided
        location = None
        if 'bearing' in request.json:
            bearing = float(request.json['bearing'])
            user_x, user_y = estimate_location(building_name, distance, bearing)
            location = {'x': user_x, 'y': user_y}
        
        # Prepare response with all data
        response = {
            'success': True,
            'recognition': {
                'building_name': building_name,
                'confidence': confidence
            },
            'building_details': {
                'reference_height': REFERENCE_SIZES[building_name]['height'],
                'reference_width': REFERENCE_SIZES[building_name]['width'],
                'pixel_height': pixel_height,
                'bounding_box': {
                    'x': bbox[0],
                    'y': bbox[1],
                    'width': bbox[2],
                    'height': bbox[3]
                }
            },
            'distance': {
                'value': distance,
                'unit': 'meters'
            }
        }
        
        if location:
            response['location'] = location
            response['building_coordinates'] = BUILDING_COORDINATES[building_name]
        print(response)
        return jsonify(response)
    
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 400


if __name__ == '__main__':
    # Load models at startup
    recognition_model, label_encoder, base_model = load_recognition_model()
    app.run(host='0.0.0.0', port=5000, debug=False) 

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.7:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:192.168.1.7 - - [03/May/2025 18:13:39] "GET / HTTP/1.1" 404 -
